# Import Necessary Libraries

In [ ]:
# Data Manipulation and Linear Algebra
import pandas as pd
import numpy as np

# Plots
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler

from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

## Getting the Data 

In [ ]:
df = pd.read_csv("../input/churn-for-bank-customers/churn.csv")
df

## Dropping some Columns of Less Importance

In [ ]:
drop_cols = ["RowNumber", "CustomerId", "Surname"]
df.drop(columns=drop_cols, inplace=True)

In [ ]:
df.info() # Luckiley we dont have any null values

In [ ]:
df.describe()

# EDA

## Plotting Correlation Heatmap Before Encoding Categorical Variables

In [ ]:
plt.figure(dpi=80, figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap="YlGnBu") # Not many correlations between features
plt.show()

## Distribution Plots

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)

plt.subplot(221)
sns.histplot(df["Age"], kde=True) # Age is Right Skewed and most of the people are around 30 - 40 years old

plt.subplot(222)
sns.histplot(df["EstimatedSalary"], kde=True) # Salary is Well Distributed

plt.subplot(223)
sns.histplot(df["CreditScore"], kde=True) # Credit Score is Left Skewed and most people have 600 - 700 Credit Score

plt.subplot(224)
sns.histplot(df["Balance"], kde=True) # Balance Data is also well distributed if 0 is ignored

## Age vs IsActiveMember, Exited

In [ ]:
plt.figure(figsize=(10, 6))

plt.subplot(121)
sns.boxplot(x="IsActiveMember", y="Age", data=df)
plt.title("IsActiveMember vs Age", fontdict={"fontsize": 14, "fontweight":500})

plt.subplot(122)
sns.boxplot(x="Exited", y="Age", data=df)
plt.title("Exited vs Age", fontdict={"fontsize": 14, "fontweight":500})

# People of Higher age i.e., 40 years and above tend to leave the bank more

## Balance vs Exited

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(121)
sns.boxplot(x="Exited", y="Balance", data=df)
plt.title("Exited vs Balance", fontdict={"fontsize": 14, "fontweight":500})
# If we observe Median value people with Higher balance tend to leave bank

plt.subplot(122)
sns.violinplot(x="Exited", y="Balance", hue="Gender", data=df)
plt.title("Exited vs Balance", fontdict={"fontsize": 14, "fontweight":500})

## NumOfProducts vs Balance

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x="NumOfProducts", y="Balance", data=df)
plt.title("NumOfProducts vs Balance", fontdict={"fontsize": 14, "fontweight":500})
# Most of the people have 1 or 4 products

## Pair Plot

In [ ]:
sns.pairplot(df, hue="Gender")
plt.show()

# Preparing Data

## OneHotEncoding Data

In [ ]:
df = pd.get_dummies(df)
df.reset_index(drop=True, inplace=True)
df

## Stratified Train Test Split
### Evenly Spreading the Dependent Variable "status" in Train and Test set

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['Exited']):
    train = df.loc[train_index]
    test = df.loc[test_index]

In [ ]:
X_train = train.drop("Exited", axis=1)
y_train = train["Exited"]

X_test = test.drop("Exited", axis=1)
y_test = test["Exited"]

## Feature Scaling

In [ ]:
cols_to_scale = ["CreditScore", "Age", "Balance", "EstimatedSalary"]

sc = StandardScaler()

X_train[cols_to_scale] = sc.fit_transform(X_train[cols_to_scale])
X_test[cols_to_scale] = sc.transform(X_test[cols_to_scale])

# Machine Learning - Multiple Model Testing

In [ ]:
# Dataframe to store all the accuracy scores for Comparison and Analysis
MLA_compare = pd.DataFrame()

def MLA_testing(MLA, X_train, X_test):
    row_index = 0
    for classifier in MLA:
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        classifier_accuracy_score = accuracy_score(y_test, y_pred)

        kfold_accuracy = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

        MLA_name = classifier.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'Accuracy Score'] = classifier_accuracy_score*100
        MLA_compare.loc[row_index, 'K-Fold Accuracy'] = kfold_accuracy.mean()*100

        print(MLA_name, "Done")
        row_index+=1

In [ ]:
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),

    XGBClassifier(eval_metric="logloss", use_label_encoder=False),
    CatBoostClassifier(silent=True)  
    ]

MLA_testing(MLA=MLA, X_train=X_train, X_test=X_test)

# Comparing Models

## Top 10 Best Performing Models

In [ ]:
MLA_compare = MLA_compare.sort_values(by="K-Fold Accuracy", ascending=False).reset_index(drop=True)
MLA_compare

# Trying Some Best Performing Models

In [ ]:
def MLA_testing(MLA, X_train, X_test, y_train, y_test):      
    # Training The Model
    MLA.fit(X_train, y_train)

    # KFold Accuracies on Training Data
    kfold_accuracy = cross_val_score(estimator = MLA, X = X_train, y = y_train, cv = 10, n_jobs=-1)
    print("K-Fold Accuracies:\n", kfold_accuracy, "\n")
    
    # Prediction on Testing Data
    y_pred = cross_val_predict(estimator = MLA, X = X_test, y = y_test, cv = 10, n_jobs=-1)
    
    # Accuracy for y_test and y_pred
    classifier_accuracy_score = accuracy_score(y_test, y_pred)
    print("Accuracy Score:\n", classifier_accuracy_score, "\n")
    
    # Confusion Matrix
    conf_mtx = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", conf_mtx, "\n")
    
    # Classification Report
    class_rep = classification_report(y_test, y_pred)
    print("Classification Report:\n", class_rep, "\n")
    
    try:
        # Precision - Recall Curve
        yhat = MLA.predict_proba(X_test)
        precision, recall, _ = precision_recall_curve(y_test, yhat[:, 1])
        
        plt.figure(dpi=100, figsize=(15, 6))
        plt.subplot(121)
        sns.lineplot([0, 1], [1, 0], linestyle='--', label='No Skill')
        sns.lineplot(recall, precision, marker='.', label=MLA.__class__.__name__)
        plt.title("Recall vs Precision Curve")
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend()
        
        # ROC Curve
        plt.subplot(122)
        sns.lineplot([0, 1], [0, 1], linestyle='--', label='No Skill')
        fpr, tpr, _ = roc_curve(y_test, yhat[:, 1])
        sns.lineplot(fpr, tpr, marker='.', label=MLA.__class__.__name__)
        plt.title("ROC Curve")
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        
        plt.show()
    except:
        pass
    
    # Important Features for The Algorithms
    imp_cols = pd.DataFrame()
    imp_cols["Features"] = X_train.columns
    imp_cols["Importance"] = MLA.feature_importances_
    imp_cols = imp_cols.sort_values(by="Importance", ascending=False)
    
    plt.figure(dpi=80, figsize=(10, 8))
    sns.barplot(y="Features", x="Importance", data=imp_cols)
    plt.title("Importance of Features")
    plt.show()

## Catboot Classifier

In [ ]:
cat_clf = CatBoostClassifier(silent=True)  

MLA_testing(cat_clf, X_train, X_test, y_train, y_test)

## GradientBoostingClassifier

In [ ]:
gb_clf = ensemble.GradientBoostingClassifier()

MLA_testing(gb_clf, X_train, X_test, y_train, y_test)

## RandomForestClassifier

In [ ]:
rf_clf = ensemble.RandomForestClassifier()

MLA_testing(rf_clf, X_train, X_test, y_train, y_test)